<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/dacon_timeseries/blob/main/final_model/%EC%8B%9C%EA%B3%84%EC%97%B4_1%ED%8C%80_private_inference_%EC%9D%BC%EC%82%AC%EC%9D%BC%EC%A1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

test 데이터 전처리 후 inference
=> ulsan_weather_06, dangjin_weather_06 최신 업데이트 버전으로 가져오기

* 이미지 한 장이 하루여야 함
* 72시간의 데이터를 갖고 73번째 시간대의 데이터를 예측하는것

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm import tqdm
from google.colab import drive
from datetime import datetime, timedelta

import math
import time
import random

import lightgbm as lgb
from sklearn.model_selection import cross_val_score, train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.externals import joblib

drive.mount('./gdrive', force_remount=True)
plt.style.use("fivethirtyeight")
plt.rc('font', family='NanumBarunGothic')
plt.rc('axes', unicode_minus=False)
warnings.filterwarnings(action='ignore')

Mounted at ./gdrive


In [3]:
dangjin_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin_weather_06.csv", encoding = 'utf-8')
ulsan_weather_06 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan_weather_06.csv", encoding = 'utf-8')
#dangjin_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/dangjin0702_0703_sunlight.csv", encoding = 'utf-8')
#ulsan_sun = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/sun/ulsan0702_0703_sunlight.csv", encoding = 'utf-8')

In [4]:
dangjin_weather_06.shape

(215, 14)

In [5]:
def interpolatee(dangjin_fcst):
  dangjin_fcst['Temperature'] = dangjin_fcst['Temperature'].interpolate()  #interpolate() 구간 따로 안정하고 column 단위로 통째로 적용하면 되는거 맞음
  #dangjin_fcst['PrecipitationForm'] = dangjin_fcst['PrecipitationForm'].interpolate()
  #dangjin_fcst['PrecipitationProb'] = dangjin_fcst['PrecipitationProb'].interpolate()
  dangjin_fcst['Humidity'] = dangjin_fcst['Humidity'].interpolate()
  dangjin_fcst['WindSpeed'] = dangjin_fcst['WindSpeed'].interpolate()
  dangjin_fcst['Cloud'] = dangjin_fcst['Cloud'].interpolate()
  #dangjin_fcst['Precipitation'] = dangjin_fcst['Precipitation'].interpolate()
  #dangjin_fcst['Snow'] = dangjin_fcst['Snow'].interpolate()
  return dangjin_fcst

In [6]:
dangjin_weather_06 = interpolatee(dangjin_weather_06)
ulsan_weather_06 = interpolatee(ulsan_weather_06)

In [7]:
#Wind Direction 항목 angle에서 방향표시로 바꿔주기
def angle_to_dir(x):
    if pd.isnull(x):
        return np.nan
    elif x >= 22.5 and x <67.5:
        return 'NE'
    elif x >= 67.5 and x < 112.5:
        return 'E'
    elif x >= 112.5 and x < 157.5:
        return 'SE'
    elif x >= 157.5 and x < 202.5:
        return 'S'
    elif x >= 202.5 and x < 247.5:
        return 'WS'
    elif x >= 247.5 and x < 292.5:
        return 'W'
    elif x >= 292.5 and x < 337.5:
        return 'NW'
    elif x >= 337.5 or x < 22.5:
        return 'N'

def angle_to_cos(x):
    return np.cos(np.pi/180*(x-90))
def angle_to_sin(x):
    return np.sin(np.pi/180*(x+90))


In [8]:
dangjin_weather_06['WindDirection_x'] = dangjin_weather_06['WindDirection'].apply(angle_to_cos)  
dangjin_weather_06['WindDirection_y'] = dangjin_weather_06['WindDirection'].apply(angle_to_sin)
dangjin_weather_06['Wind_x'] = dangjin_weather_06['WindDirection_x'] * dangjin_weather_06['WindSpeed']
dangjin_weather_06['Wind_y'] = dangjin_weather_06['WindDirection_y'] * dangjin_weather_06['WindSpeed']

ulsan_weather_06['WindDirection_x'] = ulsan_weather_06['WindDirection'].apply(angle_to_cos)
ulsan_weather_06['WindDirection_y'] = ulsan_weather_06['WindDirection'].apply(angle_to_sin)
ulsan_weather_06['Wind_x'] = ulsan_weather_06['WindDirection_x'] * ulsan_weather_06['WindSpeed']
ulsan_weather_06['Wind_y'] = ulsan_weather_06['WindDirection_y'] * ulsan_weather_06['WindSpeed']

dangjin_weather_06['WindDirection'] = dangjin_weather_06['WindDirection'].interpolate()
ulsan_weather_06['WindDirection'] = ulsan_weather_06['WindDirection'].interpolate()

#dangjin_weather_06['WindDirection_catg'] = dangjin_weather_06['WindDirection'].apply(angle_to_dir)
#ulsan_weather_06['WindDirection_catg'] = ulsan_weather_06['WindDirection'].apply(angle_to_dir)

In [9]:
dangjin_weather_06['Wind_x'] = dangjin_weather_06['Wind_x'].interpolate()
dangjin_weather_06['Wind_y'] = dangjin_weather_06['Wind_y'].interpolate()

ulsan_weather_06['Wind_x'] = ulsan_weather_06['Wind_x'].interpolate()
ulsan_weather_06['Wind_y'] = ulsan_weather_06['Wind_y'].interpolate()

In [10]:
dangjin_weather_06['hour'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.hour
dangjin_weather_06['day'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.day
dangjin_weather_06['month'] = pd.to_datetime(dangjin_weather_06['Forecast_time']).dt.month

ulsan_weather_06['hour'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.hour
ulsan_weather_06['day'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.day
ulsan_weather_06['month'] = pd.to_datetime(ulsan_weather_06['Forecast_time']).dt.month

In [11]:
#이슬점 직접 계산해서 추가
b = 17.62
c = 243.12
gamma = (b * dangjin_weather_06['Temperature'] /(c + dangjin_weather_06['Temperature'])) + np.log(dangjin_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
dangjin_weather_06['DewPoint'] = dewpoint  

gamma = (b *ulsan_weather_06['Temperature'] /(c + ulsan_weather_06['Temperature'])) + np.log(ulsan_weather_06['Humidity'] / 100.0)
dewpoint = (c * gamma) / (b - gamma)
ulsan_weather_06['DewPoint'] = dewpoint

In [12]:
dangjin_weather_06['방위각'] = dangjin_weather_06['방위각'].apply(lambda x : int(x[:3]))
dangjin_weather_06['적경'] = dangjin_weather_06['적경'].apply(lambda x : int(x.split(" ")[0]))
dangjin_weather_06['고도'] = dangjin_weather_06['고도'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
dangjin_weather_06['적위'] = dangjin_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

ulsan_weather_06['방위각'] = ulsan_weather_06['방위각'].apply(lambda x : int(x[:3]))
ulsan_weather_06['고도'] = ulsan_weather_06['고도'].apply(lambda x : int(x.split(" ")[0]))
ulsan_weather_06['적경'] = ulsan_weather_06['적경'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))
ulsan_weather_06['적위'] = ulsan_weather_06['적위'].apply(lambda x : float(x.split(" ")[0] + "." + x.split(" ")[1] + x.split(" ")[2][:2]))

In [13]:
using_col = ['Temperature', 'DewPoint', 'Humidity', 'Cloud', 'Wind_x', 'Wind_y',
             'hour', 'month', '방위각',	'고도',	'적경',	'적위','일조','일사']

dangjin_weather_06 = dangjin_weather_06[using_col].fillna(0)
ulsan_weather_06 = ulsan_weather_06[using_col].fillna(0)

In [14]:
# Shift => period만큼 데이터 옮겨서 concat
# 해당 시간대에 대해 당일 예보, 하루 전 예보, 이틀 전 예보 이렇게 column을 늘려줌
def shift(df, period):
  target = pd.DataFrame()
  for i in range(period, -1, -1): #period부터 거꾸로 1칸씩 (71부터 0까지 출력)
    tmp = df.shift(i) #i는 period
    target = pd.concat([target, tmp], axis = 1)
  return target

In [15]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,19.800000,18.286875,91.000000,1.0,1.000000e+00,1.224647e-16,1,6,5,-29.2907,6,23.1340,0.000000,0.0
1,19.300000,18.645399,96.000000,1.0,6.123234e-18,1.000000e-01,2,6,21,-26.3929,6,23.1332,0.000000,0.0
2,19.400000,19.075384,98.000000,1.0,0.000000e+00,0.000000e+00,3,6,35,-21.0002,6,23.1324,0.000000,0.0
3,19.500000,19.175136,98.000000,5.0,0.000000e+00,0.000000e+00,4,6,46,-13.0856,6,23.1316,0.000000,0.0
4,19.500000,19.175136,98.000000,2.0,1.224647e-17,2.000000e-01,5,6,57,-3.4315,6,23.1309,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,25.333333,22.947303,86.666667,4.0,-1.093641e-01,-4.698727e+00,19,7,291,9.2249,7,22.3023,0.059823,0.0
211,24.666667,22.606025,88.333333,4.0,1.140179e-01,-4.898673e+00,20,7,299,-1.2508,7,22.3006,0.000000,0.0
212,24.000000,22.256507,90.000000,4.0,3.557580e-01,-5.087577e+00,21,7,309,-11.1508,7,22.2949,0.000000,0.0
213,24.000000,22.256507,90.000000,4.0,3.557580e-01,-5.087577e+00,22,7,321,-19.3950,7,22.2932,0.000000,0.0


In [16]:
dangjin_weather_06 = shift(dangjin_weather_06, 71) 
ulsan_weather_06 = shift(ulsan_weather_06, 71) 

In [17]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.800000,18.286875,91.000000,1.0,1.000000e+00,1.224647e-16,1,6,5,-29.2907,6,23.1340,0.000000,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.800000,18.286875,91.000000,1.0,1.000000e+00,1.224647e-16,1.0,6.0,5.0,-29.2907,6.0,23.1340,0.000000,0.000000,19.300000,18.645399,96.000000,1.0,6.123234e-18,1.000000e-01,2,6,21,-26.3929,6,23.1332,0.000000,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,91.000000,1.0,1.000000e+00,1.224647e-16,1.0,6.0,5.0,-29.2907,6.0,23.1340,0.000000,0.000000,19.300000,18.645399,96.000000,1.0,6.123234e-18,1.000000e-01,2.0,6.0,21.0,-26.3929,6.0,23.1332,0.000000,0.000000,19.400000,19.075384,98.000000,1.0,0.000000e+00,0.000000e+00,3,6,35,-21.0002,6,23.1324,0.000000,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.000000,1.0,6.123234e-18,1.000000e-01,2.0,6.0,21.0,-26.3929,6.0,23.1332,0.000000,0.000000,19.400000,19.075384,98.000000,1.0,0.000000e+00,0.000000e+00,3.0,6.0,35.0,-21.0002,6.0,23.1324,0.000000,0.000000,19.500000,19.175136,98.000000,5.0,0.000000e+00,0.000000e+00,4,6,46,-13.0856,6,23.1316,0.000000,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.000000,1.0,0.000000e+00,0.000000e+00,3.0,6.0,35.0,-21.0002,6.0,23.1324,0.000000,0.000000,19.500000,19.175136,98.000000,5.0,0.000000e+00,0.000000e+00,4.0,6.0,46.0,-13.0856,6.0,23.1316,0.000000,0.000000,19.500000,19.175136,98.000000,2.0,1.224647e-17,2.000000e-01,5,6,57,-3.4315,6,23.1309,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,21.0,19.649112,92.0,7.0,-2.298133,-1.928363,20.0,7.0,300.0,-1.1806,6.0,22.4826,0.0,0.07,20.8,19.451152,92.0,5.0,-0.649838,-1.785416,21.0,7.0,310.0,-11.0556,6.0,22.4812,0.0,0.0,20.7,19.526148,93.0,9.0,-0.513030,-1.409539,22.0,7.0,321.0,-19.2802,6.0,22.4757,...,81.666667,4.0,-1.270033e-01,-4.364819e+00,17.0,7.0,274.0,32.3741,7.0,22.3058,0.041151,0.077701,26.000000,23.280070,85.000000,4.0,-3.139041e-01,-4.489038e+00,18.0,7.0,283.0,20.4851,7.0,22.3041,0.065759,0.032413,25.333333,22.947303,86.666667,4.0,-1.093641e-01,-4.698727e+00,19,7,291,9.2249,7,22.3023,0.059823,0.0
211,20.8,19.451152,92.0,5.0,-0.649838,-1.785416,21.0,7.0,310.0,-11.0556,6.0,22.4812,0.0,0.00,20.7,19.526148,93.0,9.0,-0.513030,-1.409539,22.0,7.0,321.0,-19.2802,6.0,22.4757,0.0,0.0,20.7,19.698490,94.0,8.0,-0.615636,-1.691447,23.0,7.0,334.0,-25.4845,6.0,22.4743,...,85.000000,4.0,-3.139041e-01,-4.489038e+00,18.0,7.0,283.0,20.4851,7.0,22.3041,0.065759,0.032413,25.333333,22.947303,86.666667,4.0,-1.093641e-01,-4.698727e+00,19.0,7.0,291.0,9.224

In [18]:
#앞에 72개는 nan 제거
'''
dangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]
ulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]
'''

'\ndangjin_weather_06 = dangjin_weather_06[(dangjin_weather_06.isnull().sum(axis = 1) <= 0).values]\nulsan_weather_06 = ulsan_weather_06[(ulsan_weather_06.isnull().sum(axis = 1) <= 0).values]\n'

In [19]:
dangjin_weather_06

,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,...,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사,Temperature,DewPoint,Humidity,Cloud,Wind_x,Wind_y,hour,month,방위각,고도,적경,적위,일조,일사
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.800000,18.286875,91.000000,1.0,1.000000e+00,1.224647e-16,1,6,5,-29.2907,6,23.1340,0.000000,0.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19.800000,18.286875,91.000000,1.0,1.000000e+00,1.224647e-16,1.0,6.0,5.0,-29.2907,6.0,23.1340,0.000000,0.000000,19.300000,18.645399,96.000000,1.0,6.123234e-18,1.000000e-01,2,6,21,-26.3929,6,23.1332,0.000000,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,91.000000,1.0,1.000000e+00,1.224647e-16,1.0,6.0,5.0,-29.2907,6.0,23.1340,0.000000,0.000000,19.300000,18.645399,96.000000,1.0,6.123234e-18,1.000000e-01,2.0,6.0,21.0,-26.3929,6.0,23.1332,0.000000,0.000000,19.400000,19.075384,98.000000,1.0,0.000000e+00,0.000000e+00,3,6,35,-21.0002,6,23.1324,0.000000,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,96.000000,1.0,6.123234e-18,1.000000e-01,2.0,6.0,21.0,-26.3929,6.0,23.1332,0.000000,0.000000,19.400000,19.075384,98.000000,1.0,0.000000e+00,0.000000e+00,3.0,6.0,35.0,-21.0002,6.0,23.1324,0.000000,0.000000,19.500000,19.175136,98.000000,5.0,0.000000e+00,0.000000e+00,4,6,46,-13.0856,6,23.1316,0.000000,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,98.000000,1.0,0.000000e+00,0.000000e+00,3.0,6.0,35.0,-21.0002,6.0,23.1324,0.000000,0.000000,19.500000,19.175136,98.000000,5.0,0.000000e+00,0.000000e+00,4.0,6.0,46.0,-13.0856,6.0,23.1316,0.000000,0.000000,19.500000,19.175136,98.000000,2.0,1.224647e-17,2.000000e-01,5,6,57,-3.4315,6,23.1309,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,21.0,19.649112,92.0,7.0,-2.298133,-1.928363,20.0,7.0,300.0,-1.1806,6.0,22.4826,0.0,0.07,20.8,19.451152,92.0,5.0,-0.649838,-1.785416,21.0,7.0,310.0,-11.0556,6.0,22.4812,0.0,0.0,20.7,19.526148,93.0,9.0,-0.513030,-1.409539,22.0,7.0,321.0,-19.2802,6.0,22.4757,...,81.666667,4.0,-1.270033e-01,-4.364819e+00,17.0,7.0,274.0,32.3741,7.0,22.3058,0.041151,0.077701,26.000000,23.280070,85.000000,4.0,-3.139041e-01,-4.489038e+00,18.0,7.0,283.0,20.4851,7.0,22.3041,0.065759,0.032413,25.333333,22.947303,86.666667,4.0,-1.093641e-01,-4.698727e+00,19,7,291,9.2249,7,22.3023,0.059823,0.0
211,20.8,19.451152,92.0,5.0,-0.649838,-1.785416,21.0,7.0,310.0,-11.0556,6.0,22.4812,0.0,0.00,20.7,19.526148,93.0,9.0,-0.513030,-1.409539,22.0,7.0,321.0,-19.2802,6.0,22.4757,0.0,0.0,20.7,19.698490,94.0,8.0,-0.615636,-1.691447,23.0,7.0,334.0,-25.4845,6.0,22.4743,...,85.000000,4.0,-3.139041e-01,-4.489038e+00,18.0,7.0,283.0,20.4851,7.0,22.3041,0.065759,0.032413,25.333333,22.947303,86.666667,4.0,-1.093641e-01,-4.698727e+00,19.0,7.0,291.0,9.224

In [20]:
#min max scaling
dangjin_min = dangjin_weather_06.min()
dangjin_max = dangjin_weather_06.max()

dangjin_weather_06 = (dangjin_weather_06 - dangjin_min) / (dangjin_max - dangjin_min)

In [21]:
ulsan_min = ulsan_weather_06.min()
ulsan_max = ulsan_weather_06.max()

ulsan_weather_06 = (ulsan_weather_06 - ulsan_min) / (ulsan_max - ulsan_min)

# 2D Conv 모델

In [22]:
import tensorflow as tf

In [23]:
dangjin_weather_06.shape

(215, 1008)

In [24]:
dangjin_weather_06 = dangjin_weather_06.values.reshape((dangjin_weather_06.shape[0], 72, 14, 1)) #(193, 864) -> (193, 72, 12, 1)
ulsan_weather_06 = ulsan_weather_06.values.reshape((ulsan_weather_06.shape[0], 72, 14, 1))

In [25]:
dangjin_weather_06.shape

(215, 72, 14, 1)

In [26]:
'''
dangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) 
ulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))
'''

'\ndangjin_weather_06 = tf.image.resize(dangjin_weather_06, (72, 32)) \nulsan_weather_06 = tf.image.resize(ulsan_weather_06, (72, 32))\n'

In [27]:
dangjin_weather_06 = np.squeeze(dangjin_weather_06, axis = -1) 
dangjin_weather_06 = np.stack((dangjin_weather_06,)*3, axis = -1)  

ulsan_weather_06 = np.squeeze(ulsan_weather_06, axis = -1)
ulsan_weather_06 = np.stack((ulsan_weather_06,)*3, axis = -1)

In [28]:
ulsan_weather_06.shape

(215, 72, 14, 3)

In [29]:
dangjin_test = dangjin_weather_06[-24:]
ulsan_test = ulsan_weather_06[-24:]

In [30]:
dangjin_test.shape

(24, 72, 14, 3)

* Inference

In [31]:
floating_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold0.h5")
floating_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold1.h5")
floating_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold2.h5")
floating_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold3.h5")
floating_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/floating_fold4.h5")

In [32]:
floating0 = floating_model0.predict(dangjin_test)
floating1 = floating_model1.predict(dangjin_test)
floating2 = floating_model2.predict(dangjin_test)
floating3 = floating_model3.predict(dangjin_test)
floating4 = floating_model4.predict(dangjin_test)

In [33]:
floating2

array([[  1.4739119 ],
       [  1.1726309 ],
       [  1.081013  ],
       [  1.2379464 ],
       [  1.4697208 ],
       [  1.8249741 ],
       [  2.1625721 ],
       [  2.5279498 ],
       [  2.82704   ],
       [ 10.203307  ],
       [ 46.79886   ],
       [123.0398    ],
       [214.6634    ],
       [305.27225   ],
       [340.20035   ],
       [229.77142   ],
       [ 76.18904   ],
       [ 18.379602  ],
       [  3.512222  ],
       [  0.7282146 ],
       [  0.7494192 ],
       [  0.8075206 ],
       [  0.86100626],
       [  1.134814  ]], dtype=float32)

In [34]:
warehouse_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold0.h5")
warehouse_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold1.h5")
warehouse_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold2.h5")
warehouse_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold3.h5")
warehouse_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/warehouse_fold4.h5")

In [35]:
warehouse0 = warehouse_model0.predict(dangjin_test)
warehouse1 = warehouse_model1.predict(dangjin_test)
warehouse2 = warehouse_model2.predict(dangjin_test)
warehouse3 = warehouse_model3.predict(dangjin_test)
warehouse4 = warehouse_model4.predict(dangjin_test)

In [36]:
dangjin_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold0.h5")
dangjin_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold1.h5")
dangjin_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold2.h5")
dangjin_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold3.h5")
dangjin_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/dangjin_fold4.h5")

In [37]:
dangjin0 = dangjin_model0.predict(dangjin_test)
dangjin1 = dangjin_model1.predict(dangjin_test)
dangjin2 = dangjin_model2.predict(dangjin_test)
dangjin3 = dangjin_model3.predict(dangjin_test)
dangjin4 = dangjin_model4.predict(dangjin_test)

In [38]:
ulsan_model0 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold0.h5")
ulsan_model1 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold1.h5")
ulsan_model2 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold2.h5")
ulsan_model3 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold3.h5")
ulsan_model4 = tf.keras.models.load_model("/content/gdrive/MyDrive/dacon_timeseries/final_model/model/ulsan_fold4.h5")

In [39]:
dangjin_test.shape

(24, 72, 14, 3)

In [40]:
ulsan_test.shape

(24, 72, 14, 3)

In [41]:
ulsan0 = ulsan_model0.predict(ulsan_test)
ulsan1 = ulsan_model1.predict(ulsan_test)
ulsan2 = ulsan_model2.predict(ulsan_test)
ulsan3 = ulsan_model3.predict(ulsan_test)
ulsan4 = ulsan_model4.predict(ulsan_test)

In [42]:

floating = (floating0 + floating1 + floating2 + floating3 + floating4) / 5
warehouse = (warehouse0 + warehouse1 + warehouse2 + warehouse3 + warehouse4 ) / 5
dangjin = (dangjin0 + dangjin1 + dangjin2 + dangjin3 + dangjin4) / 5
ulsan = (ulsan0 + ulsan1 + ulsan2 + ulsan3 + ulsan4) / 5


In [43]:
'''
floating = (floating0 + floating1 + floating2) / 3
warehouse = (warehouse0 + warehouse1 + warehouse2 ) / 3
dangjin = (dangjin0 + dangjin1 + dangjin2) / 3
ulsan = (ulsan0 + ulsan1 + ulsan2 ) / 3
'''

'\nfloating = (floating0 + floating1 + floating2) / 3\nwarehouse = (warehouse0 + warehouse1 + warehouse2 ) / 3\ndangjin = (dangjin0 + dangjin1 + dangjin2) / 3\nulsan = (ulsan0 + ulsan1 + ulsan2 ) / 3\n'

In [44]:
#sample_submission2 = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/sample_submission.csv")
sample_submission = pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/private_data/neural_prophet_validation.csv")

In [45]:
#sample_submission2

In [ ]:
sample_submission

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.000000,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.000000,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.000000,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.000000,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.000000,0.0,0.651203
...,...,...,...,...,...
1387,2021-07-08 20:00:00,0.0,1.125614,0.0,2.495743
1388,2021-07-08 21:00:00,0.0,1.879234,0.0,3.404133
1389,2021-07-08 22:00:00,0.0,0.933403,0.0,4.150723
1390,2021-07-08 23:00:00,0.0,0.127186,0.0,3.220158


* 시간은 내일 00시부터 23시까지 !! (가져온 날씨 데이터가 그럼)

In [47]:
#loc는 끝 인덱스도 포함....
#sample_submission.loc[815:839] #6/14일 제출
#sample_submission.loc[839:862] #6/15일
#sample_submission.loc[863:886] #6/16일
#sample_submission.loc[887:910] #6/17일
#sample_submission.loc[911:934] #6/18일
#sample_submission.loc[935:958] #6/19일
#sample_submission.loc[959:982] #6/20일
#sample_submission.loc[983:1006] #6/20일
#sample_submission.loc[1295:1318] #7/4
#sample_submission.loc[1319:1342]
sample_submission.loc[1343:1366]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1343,2021-07-06 24:00:00,0.000000,8.891773,0.000000,6.607231
1344,2021-07-07 01:00:00,0.000000,10.251427,2.448341,7.010326
1345,2021-07-07 02:00:00,0.000000,10.018150,1.226212,6.532532
1346,2021-07-07 03:00:00,0.000000,10.856544,1.451569,5.472900
1347,2021-07-07 04:00:00,0.000000,12.479786,6.691200,6.204327
1348,2021-07-07 05:00:00,0.000000,11.422241,9.763237,7.121971
1349,2021-07-07 06:00:00,0.000000,8.872986,6.014053,5.377926
1350,2021-07-07 07:00:00,0.000000,14.376915,9.396729,8.531662
1351,2021-07-07 08:00:00,26.605057,41.395439,46.415253,32.962406
1352,2021-07-07 09:00:00,94.269363,98.537460,132.831161,83.609932


In [ ]:
floating

array([[  2.4601412],
       [  2.5823867],
       [  2.5504615],
       [  2.43695  ],
       [  2.444633 ],
       [  2.4066174],
       [  2.8081899],
       [  5.6439066],
       [ 20.768415 ],
       [ 94.70459  ],
       [198.16928  ],
       [270.14468  ],
       [327.27716  ],
       [312.74536  ],
       [273.16754  ],
       [229.05562  ],
       [167.22116  ],
       [110.38139  ],
       [ 63.569733 ],
       [ 14.143152 ],
       [  2.0237184],
       [  1.4589626],
       [  1.5775397],
       [  1.883469 ]], dtype=float32)

In [ ]:
warehouse

array([[  1.8233902],
       [  1.7961569],
       [  1.9179659],
       [  2.1867037],
       [  2.3338091],
       [  2.3344638],
       [  2.3360944],
       [  2.5504143],
       [  7.017029 ],
       [ 32.971817 ],
       [ 77.30202  ],
       [113.95321  ],
       [148.9844   ],
       [146.87657  ],
       [128.82712  ],
       [106.25604  ],
       [ 81.41909  ],
       [ 48.654377 ],
       [ 19.418142 ],
       [  2.9357414],
       [  1.699897 ],
       [  1.7070005],
       [  1.7396553],
       [  1.9013088]], dtype=float32)

In [ ]:
floating.shape

(24, 1)

In [ ]:
sample_submission.loc[1319:1342].shape

(24, 5)

In [48]:
sample_submission.loc[1343:1366, 'dangjin_floating'] = floating
sample_submission.loc[1343:1366, 'dangjin_warehouse'] = warehouse
sample_submission.loc[1343:1366, 'dangjin'] = dangjin
sample_submission.loc[1343:1366, 'ulsan'] = ulsan

In [49]:
sample_submission.loc[1343:1366]

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
1343,2021-07-06 24:00:00,0.864309,1.530082,6.188235,126.312035
1344,2021-07-07 01:00:00,0.784268,1.427142,5.246865,126.312035
1345,2021-07-07 02:00:00,0.796252,1.502519,4.510379,126.312035
1346,2021-07-07 03:00:00,0.837645,1.661959,4.066080,126.312035
1347,2021-07-07 04:00:00,0.933640,1.793231,3.801616,126.312035
1348,2021-07-07 05:00:00,1.158926,1.897506,4.763043,126.312035
1349,2021-07-07 06:00:00,1.555002,2.009407,5.289058,126.312035
1350,2021-07-07 07:00:00,2.190744,2.083850,6.336852,126.312035
1351,2021-07-07 08:00:00,3.235486,2.908154,7.185033,126.312035
1352,2021-07-07 09:00:00,17.979298,11.959776,28.891903,126.312035


In [51]:
sample_submission.to_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0706_submission.csv", index = False)

In [ ]:
ls

In [53]:
pd.read_csv("/content/gdrive/MyDrive/dacon_timeseries/final_model/0706_submission.csv")

,time,dangjin_floating,dangjin_warehouse,dangjin,ulsan
0,2021-02-01 01:00:00,0.0,0.0,0.0,0.000000
1,2021-02-01 02:00:00,0.0,0.0,0.0,0.000000
2,2021-02-01 03:00:00,0.0,0.0,0.0,0.000000
3,2021-02-01 04:00:00,0.0,0.0,0.0,0.000000
4,2021-02-01 05:00:00,0.0,0.0,0.0,0.651203
...,...,...,...,...,...
95,2021-02-04 24:00:00,0.0,0.0,0.0,2.694244
96,2021-02-05 01:00:00,0.0,0.0,0.0,3.571945
97,2021-02-05 02:00:00,0.0,0.0,0.0,3.577793
98,2021-02-05 03:00:00,0.0,0.0,0.0,3.007526
